# Задача 1: сравнение предложений
Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

кошки (животные)
UNIX-утилита cat для вывода содержимого файлов
версии операционной системы OS X, названные в честь семейства кошачьих
Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

Выполните следующие шаги:

1) Скачайте файл с предложениями (sentences.txt).
2) Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
3) Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
4) Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
5) Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
6) Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
7) Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).
8) Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.

In [150]:
import pandas as pd
import numpy as np
import scipy as sc
from scipy.spatial import distance
import re 

# Решение:
1) Скачайте файл с предложениями (sentences.txt).
2) Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().

In [151]:
file_words = open('sentences.txt','r')
content = file_words.readlines()
content = [x.strip().lower() for x in content]
print(content)

['in comparison to dogs, cats have not undergone major changes during the domestication process.', 'as cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.', 'a common interactive use of cat for a single file is to output the content of a file to standard output.', 'cats can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small animals.', 'in one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.', 'the domesticated cat and its closest wild ancestor are both diploid organisms that possess 38 chromosomes and roughly 20,000 genes.', 'domestic cats are similar in size to the other members of the genus felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).', 'however, if the output is piped or redirected, cat is unnecessary.', 'cat with one named file is safer where human error is a conc

# Решение:
3) Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.

In [152]:
words = []
set_words = set()
for line in content:
    line = re.split('[^a-z]', line)
    articles = []
    for article in line:
        if article != '':
            articles.append(article)
            set_words.add(article)   
    words.append(articles)
    #print(articles)
#print(len(set_words))
print(words[3])

['cats', 'can', 'hear', 'sounds', 'too', 'faint', 'or', 'too', 'high', 'in', 'frequency', 'for', 'human', 'ears', 'such', 'as', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals']


# Решение:
4) Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.

In [153]:
list_words = list(set_words)
dict_words = {x: list_words[x] for x in range(len(set_words))}
print(dict_words)
#print(dict_words.values('a'))

{0: 'they', 1: 'command', 2: 'wild', 3: 'domestic', 4: 'online', 5: 'marks', 6: 'download', 7: 'redirected', 8: 'editions', 9: 'as', 10: 'such', 11: 'artificial', 12: 'starting', 13: 'leopard', 14: 'changes', 15: 'no', 16: 'useful', 17: 'right', 18: 'be', 19: 'unix', 20: 'was', 21: 'high', 22: 'default', 23: 'ears', 24: 'on', 25: 'commands', 26: 'allows', 27: 'catenates', 28: 'part', 29: 'over', 30: 'streams', 31: 'other', 32: 'also', 33: 'are', 34: 'predecessor', 35: 'keyboards', 36: 'small', 37: 'undergone', 38: 'purchase', 39: 'incremental', 40: 'stdout', 41: 'its', 42: 'safari', 43: 'two', 44: 'according', 45: 'named', 46: 'need', 47: 'chromosomes', 48: 'installation', 49: 'when', 50: 'contains', 51: 'between', 52: 'without', 53: 'version', 54: 'since', 55: 'roughly', 56: 'os', 57: 'or', 58: 'frequency', 59: 'however', 60: 'more', 61: 'genes', 62: 's', 63: 'october', 64: 'july', 65: 'allow', 66: 'output', 67: 'year', 68: 'longer', 69: 'yosemite', 70: 'organisms', 71: 'later', 72: '

# Решение:
5) Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.

In [154]:
matr = np.ones((22, 254))
print(matr.shape)
for i in range(0, 22):
    for j in range (0, 254):
        matr[i, j] = words[i].count(dict_words[j])
    #matr[i, :]
    #print(matr[i, :])
print(matr)

(22, 254)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Решение:
6) Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.

In [155]:
weight_of_importance = []
for i in range(1, 22):
    weight_of_importance.append(distance.cosine(matr[0, :],matr[i, :])) 
weight_of_importance_min = sorted(weight_of_importance)
#print(len(weight_of_importance))
#print(min(weight_of_importance))
print(weight_of_importance.index(weight_of_importance_min[0]))
print(weight_of_importance.index(weight_of_importance_min[1]))
sentences_1 = weight_of_importance.index(weight_of_importance_min[0]) + 1
sentences_2 = weight_of_importance.index(weight_of_importance_min[1]) + 1
print("Первое ближайшее предложение к первому:\n", sentences_1)
print("Второе ближайшее предложение к первому:\n", sentences_2)
#print(weight_of_importance)
#print(weight_of_importance_min)

5
3
Первое ближайшее предложение к первому:
 6
Второе ближайшее предложение к первому:
 4


# Решение:
7) Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).

In [157]:
file_submission = open("submission-1.txt", "w+")
stroka = str(sentences_1) + ' ' + str(sentences_2)
print(stroka)
file_submission.write(stroka)
file_submission.close()

6 4


In [161]:
# Проверка файла
file_submission_2 = open("submission-1.txt", "r")
file_submission_2.readlines()

['6 4']